In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
data = spark.read.csv('SMSSpamCollection',inferSchema=True,
                     sep='\t')

In [5]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [6]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [7]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [8]:
from pyspark.sql.functions import length

In [9]:
data = data.withColumn('length',length(data['text']))

In [10]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [11]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.48663212435233|
| spam|138.6706827309237|
+-----+-----------------+



In [12]:
from pyspark.ml.feature import (Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer)

In [34]:
token = Tokenizer(inputCol='text',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_number = StringIndexer(inputCol='class',outputCol='label')

In [35]:
from pyspark.ml.feature import VectorAssembler

In [36]:
vec = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [37]:
from pyspark.ml.classification import NaiveBayes

In [38]:
nb = NaiveBayes()

In [39]:
from pyspark.ml import Pipeline

In [48]:
data_prep_pipe = Pipeline(stages= [ham_spam_to_number,token,
                                  stop_remove,count_vec,idf,vec])

In [50]:
cleaner = data_prep_pipe.fit(data).transform(data)

In [51]:
cleaner_data = cleaner.select('label','features')

In [52]:
cleaner_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13461,[8,12,33,6...|
|  0.0|(13461,[0,26,308,...|
|  1.0|(13461,[2,14,20,3...|
|  0.0|(13461,[0,73,84,1...|
|  0.0|(13461,[36,39,140...|
|  1.0|(13461,[11,57,62,...|
|  0.0|(13461,[11,55,108...|
|  0.0|(13461,[133,195,4...|
|  1.0|(13461,[1,50,124,...|
|  1.0|(13461,[0,1,14,29...|
|  0.0|(13461,[5,19,36,4...|
|  1.0|(13461,[9,18,40,9...|
|  1.0|(13461,[14,32,50,...|
|  0.0|(13461,[42,99,101...|
|  0.0|(13461,[567,1744,...|
|  1.0|(13461,[32,113,11...|
|  0.0|(13461,[86,224,37...|
|  0.0|(13461,[0,2,52,13...|
|  0.0|(13461,[0,77,107,...|
|  1.0|(13461,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



In [53]:
training, test = cleaner_data.randomSplit([0.7,0.3])

In [55]:
spam_detector = nb.fit(training)

In [56]:
results = spam_detector.transform(test)

In [57]:
results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13461,[0,1,2,44,...|[-1076.9815533176...|[1.0,4.0936940368...|       0.0|
|  0.0|(13461,[0,1,4,53,...|[-797.29335762905...|[1.0,6.7129313822...|       0.0|
|  0.0|(13461,[0,1,5,15,...|[-749.65313599275...|[1.0,6.0222986893...|       0.0|
|  0.0|(13461,[0,1,10,15...|[-537.91420585892...|[1.0,8.5423132727...|       0.0|
|  0.0|(13461,[0,1,15,33...|[-218.46502089575...|[1.0,5.8298875800...|       0.0|
|  0.0|(13461,[0,1,25,66...|[-1342.5701529305...|[1.0,4.2210480135...|       0.0|
|  0.0|(13461,[0,1,46,72...|[-634.47622336473...|[2.47475323153521...|       1.0|
|  0.0|(13461,[0,1,896,1...|[-96.909017923188...|[0.99999998816962...|       0.0|
|  0.0|(13461,[0,2,3,4,7...|[-1239.4235832749...|[1.0,1.7165451535...|       0.0|
|  0.0|(13461,[0

In [58]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [59]:
acc_eval = MulticlassClassificationEvaluator()

In [60]:
acc = acc_eval.evaluate(results)

In [61]:
print(acc)

0.9165129607191634
